In [8]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model, load_model
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import seaborn as sns
sns.set_style('darkgrid')
from sklearn.metrics import confusion_matrix, classification_report
from IPython.display import display, HTML

In [9]:
path=r'G:\Dataset-image-Eye_Data_New'
path_list=os.listdir(path)
classes=[]
file_Paths=[]
labels=[]

for i in path_list:
    class_path=os.path.join(path,i)
    if os.path.isdir(class_path):
        if i !=".git":
            classes.append(i)
class_count=len(classes)
classes

['Keratoconus', 'Normal', 'Suspect']

In [10]:
for j in classes:
    class_path=os.path.join(path,j)
    image_names=os.listdir(class_path)
    for k in image_names:
        image_path=os.path.join(class_path,k)
        file_Paths.append(image_path)
        labels.append(j)
file_Paths[:10]

['G:\\Dataset-image-Eye_Data_New\\Keratoconus\\KCN_100_CT_A - Copy - Copy - Copy.jpg',
 'G:\\Dataset-image-Eye_Data_New\\Keratoconus\\KCN_100_CT_A - Copy - Copy.jpg',
 'G:\\Dataset-image-Eye_Data_New\\Keratoconus\\KCN_100_CT_A - Copy.jpg',
 'G:\\Dataset-image-Eye_Data_New\\Keratoconus\\KCN_100_CT_A.jpg',
 'G:\\Dataset-image-Eye_Data_New\\Keratoconus\\KCN_100_EC_A - Copy - Copy - Copy.jpg',
 'G:\\Dataset-image-Eye_Data_New\\Keratoconus\\KCN_100_EC_A - Copy - Copy.jpg',
 'G:\\Dataset-image-Eye_Data_New\\Keratoconus\\KCN_100_EC_A - Copy.jpg',
 'G:\\Dataset-image-Eye_Data_New\\Keratoconus\\KCN_100_EC_A.jpg',
 'G:\\Dataset-image-Eye_Data_New\\Keratoconus\\KCN_100_EC_P - Copy - Copy - Copy.jpg',
 'G:\\Dataset-image-Eye_Data_New\\Keratoconus\\KCN_100_EC_P - Copy - Copy.jpg']

In [11]:
file_series=pd.Series(file_Paths,name='file_path')
label_series=pd.Series(labels,name='label')
data_df=pd.concat([file_series,label_series],axis=1)
print(data_df.head(10))
balance_df=data_df['label'].value_counts()
balance_df

                                           file_path        label
0  G:\Dataset-image-Eye_Data_New\Keratoconus\KCN_...  Keratoconus
1  G:\Dataset-image-Eye_Data_New\Keratoconus\KCN_...  Keratoconus
2  G:\Dataset-image-Eye_Data_New\Keratoconus\KCN_...  Keratoconus
3  G:\Dataset-image-Eye_Data_New\Keratoconus\KCN_...  Keratoconus
4  G:\Dataset-image-Eye_Data_New\Keratoconus\KCN_...  Keratoconus
5  G:\Dataset-image-Eye_Data_New\Keratoconus\KCN_...  Keratoconus
6  G:\Dataset-image-Eye_Data_New\Keratoconus\KCN_...  Keratoconus
7  G:\Dataset-image-Eye_Data_New\Keratoconus\KCN_...  Keratoconus
8  G:\Dataset-image-Eye_Data_New\Keratoconus\KCN_...  Keratoconus
9  G:\Dataset-image-Eye_Data_New\Keratoconus\KCN_...  Keratoconus


label
Normal         5600
Keratoconus    5572
Suspect        4844
Name: count, dtype: int64

In [12]:
train_split=.8
test_split=.1
dummy_split=test_split/(1-train_split)
train_df,dummy_df=train_test_split(data_df,train_size=train_split,shuffle=True,random_state=125)
test_df,valid_df=train_test_split(dummy_df,train_size=dummy_split,shuffle=True,random_state=125)
batch_size=16
length=len(test_df)
print(f"Train Size: {len(train_df)}\nTest size: {len(test_df)}\nValid size: {len(valid_df)}")


Train Size: 12812
Test size: 1602
Valid size: 1602


In [19]:
def scalar(x):
    return x/127.5-1
trgen=tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=scalar, horizontal_flip=True)
train_gen=trgen.flow_from_dataframe(train_df, x_col='file_path', y_col='label', target_size=(448,448), class_mode='categorical',batch_size=batch_size, shuffle=True, seed=123)
tvgen=tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=scalar)
valid_gen=tvgen.flow_from_dataframe(valid_df, x_col='file_path', y_col='label', target_size=(448,448), class_mode='categorical',
                                   batch_size=batch_size, shuffle=False)
test_batch_size=sorted([int(length/n) for n in range(1,length+1) if length % n ==0 and length/n<=batch_size],reverse=True)[0]
test_steps=int(length/test_batch_size)
test_gen=tvgen.flow_from_dataframe(test_df, x_col='file_path', y_col='label', target_size=(448,448), class_mode='categorical',
                                   batch_size=test_batch_size, shuffle=False)
test_labels=test_gen.labels

Found 12812 validated image filenames belonging to 3 classes.
Found 1602 validated image filenames belonging to 3 classes.
Found 1602 validated image filenames belonging to 3 classes.


In [21]:
def show_training_samples(gen):
    class_dict=gen.class_indices
    new_dict={}
    for key, value in class_dict.items(): 
        new_dict[value]=key
    images,labels=next(gen) 
    plt.figure(figsize=(15, 15))
    length=len(labels)
    if length<25: 
        r=length
    else:
        r=25
    for i in range(r):
        plt.subplot(5, 5, i + 1)
        image=(images[i]+1 )/2 
        plt.imshow(image)
        index=np.argmax(labels[i])
        class_name=new_dict[index]
        plt.title(class_name, color='blue', fontsize=16)
        plt.axis('off')
    plt.show()

In [23]:
img_shape=(448,448,3)
neurons=512
dropout=.3
lr=.001
freeze=True
base_model=tf.keras.applications.VGG19(include_top=False, input_shape=img_shape, pooling='max', weights='imagenet')
if freeze:
    base_model.trainable=False
x=base_model.output
x=tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
x =tf.keras.layers.Dense(neurons, kernel_regularizer = regularizers.l2(0.016),activity_regularizer=regularizers.l1(0.006),
                bias_regularizer=regularizers.l1(0.006) ,activation='relu', kernel_initializer= tf.keras.initializers.GlorotUniform(seed=123))(x)
x=tf.keras.layers.Dropout(rate=dropout, seed=123)(x)
output=tf.keras.layers.Dense(class_count, activation='softmax',kernel_initializer=tf.keras.initializers.GlorotUniform(seed=123))(x)
model=Model(inputs=base_model.input, outputs=output)
model.compile(Adamax(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
def print_in_color(txt_msg,fore_tupple,back_tupple,):
    rf,gf,bf=fore_tupple
    rb,gb,bb=back_tupple
    msg='{0}' + txt_msg
    mat='\33[38;2;' + str(rf) +';' + str(gf) + ';' + str(bf) + ';48;2;' + str(rb) + ';' +str(gb) + ';' + str(bb) +'m'
    print(msg .format(mat), flush=True)
    print('\33[0m', flush=True)
    return